In [ ]:
from api_neurotask import *
from baselines.metrics import get_R2
from baselines.decoders import *
from bayes_opt import BayesianOptimization, UtilityFunction

In [ ]:
parquet_file_path = 'NeuroTask/2_10_Chowdhury_CObump.parquet'
df,bin = load_and_filter_parquet(parquet_file_path,['A', 'I','F'])

In [ ]:
df10 = rebin(df,prev_bin_size = bin ,new_bin_size = 20)

In [ ]:
X_train_list, y_train_list, X_val_list, y_val_list, X_test_list, y_test_list = process_data(df10, bins_before=5, 
                                                 training_range=[0, 0.7], valid_range=[0.7, 0.8], testing_range=[0.8, 1], 
                                                 behavior_columns=['hand_vel_x', 'hand_vel_y'],zscore = True)

In [ ]:
print("There are", sum(X.shape[2] for X in X_train_list), "unique neurons in the dataset in",len(X_train_list),"different sessions")

### Wiener Filter

In [ ]:
for i in range(len(X_train_list)):
    X_flat_train=X_train_list[i].reshape(X_train_list[i].shape[0],(X_train_list[i].shape[1]*X_train_list[i].shape[2]))

    X_flat_test = X_test_list[i].reshape(X_test_list[i].shape[0],(X_test_list[i].shape[1]*X_test_list[i].shape[2]))

    #Declare model
    model_wf=WienerFilterDecoder()

    #Fit model|
    model_wf.fit(X_flat_train,y_train_list[i])

    #Get predictions
    y_valid_predicted_wf=model_wf.predict(X_flat_test)

    #Get metric of fit
    R2s_wf=get_R2(y_test_list[i],y_valid_predicted_wf)
    print('session', i+1,'R2s:', R2s_wf)

### Wiener Cascade

In [ ]:
for i in range(len(X_train_list)):
    #for i in range(len(X_train_list)):
    X_flat_train=X_train_list[i].reshape(X_train_list[i].shape[0],(X_train_list[i].shape[1]*X_train_list[i].shape[2]))

    X_flat_test = X_test_list[i].reshape(X_test_list[i].shape[0],(X_test_list[i].shape[1]*X_test_list[i].shape[2]))

    #Declare model
    model_wf=WienerCascadeDecoder(degree=3)

    #Fit model
    model_wf.fit(X_flat_train,y_train_list[i])

    #Get predictions
    y_valid_predicted_wf=model_wf.predict(X_flat_test)

    #Get metric of fit
    R2s_wf=get_R2(y_test_list[i],y_valid_predicted_wf)
    print('session', i+1,'R2s:', R2s_wf)

### DNN

Hyperparameters optimization

In [ ]:
def dnn_evaluate(num_units,frac_dropout,n_epochs):
            num_units=int(num_units)
            frac_dropout=float(frac_dropout)
            n_epochs=int(n_epochs)
            model_dnn=DenseNNDecoder(units=num_units,dropout=frac_dropout,num_epochs=n_epochs)
            model_dnn.fit(X_train,y_train)
            y_valid_predicted_dnn=model_dnn.predict(X_valid)
            return np.mean(get_R2(y_valid,y_valid_predicted_dnn))

In [ ]:
for i in range(len(X_train_list)):   
        print('session ',i+1)   

        X_train = X_train_list[i].reshape(X_train_list[i].shape[0],(X_train_list[i].shape[1]*X_train_list[i].shape[2]))
        X_valid = X_val_list[i].reshape(X_val_list[i].shape[0],(X_val_list[i].shape[1]*X_val_list[i].shape[2]))
        y_train = y_train_list[i]
        y_valid = y_val_list[i] 
        
        #Do bayesian optimization
        dnnBO = BayesianOptimization(dnn_evaluate, {'num_units': (50, 500), 'frac_dropout': (0,.5), 'n_epochs': (2,200)})
        #lstmBO.maximize(init_points=20, n_iter=20, kappa=10)
        utility = UtilityFunction(kind="ucb", kappa=10, xi=0.0)
        dnnBO.maximize(init_points=10,n_iter=5)

        best_params=dnnBO.max['params']
        frac_dropout=float(best_params['frac_dropout'])
        n_epochs=int(best_params['n_epochs'])
        num_units=int(best_params['num_units'])

Test

In [ ]:
i = 0 #session

X_flat_train=X_train_list[i].reshape(X_train_list[i].shape[0],(X_train_list[i].shape[1]*X_train_list[i].shape[2]))

X_flat_test = X_test_list[i].reshape(X_test_list[i].shape[0],(X_test_list[i].shape[1]*X_test_list[i].shape[2]))

model_dnn=DenseNNDecoder(units=200,dropout=0.25,num_epochs=5)

#Fit model
model_dnn.fit(X_flat_train,y_train_list[i])

#Get predictions
y_test_predicted_dnn=model_dnn.predict(X_flat_test)

#Get metric of fit
R2s_dnn=get_R2(y_test_list[i],y_test_predicted_dnn)
print('session', i+1,'R2s:', R2s_dnn)

### LSTM

Hyperparameters optimization

In [ ]:
def lstm_evaluate(num_units,frac_dropout,n_epochs):
            num_units=int(num_units)
            frac_dropout=float(frac_dropout)
            n_epochs=int(n_epochs)
            model_lstm=LSTMDecoder(units=num_units,dropout=frac_dropout,num_epochs=n_epochs)
            model_lstm.fit(X_train,y_train)
            y_valid_predicted_lstm=model_lstm.predict(X_valid)
            return np.mean(get_R2(y_valid,y_valid_predicted_lstm))

In [ ]:
for i in range(len(X_train_list)):
        print('session ',i+1)
        X_train = X_train_list[i]
        X_valid = X_val_list[i]
        y_train = y_train_list[i]
        y_valid = y_val_list[i]
                
        #Do bayesian optimization
        lstmBO = BayesianOptimization(lstm_evaluate, {'num_units': (50, 500), 'frac_dropout': (0,.5), 'n_epochs': (2,25)})
        #lstmBO.maximize(init_points=20, n_iter=20, kappa=10)
        utility = UtilityFunction(kind="ucb", kappa=10, xi=0.0)
        lstmBO.maximize(init_points=10,n_iter=5)

        best_params=lstmBO.max['params']
        frac_dropout=float(best_params['frac_dropout'])
        n_epochs=int(best_params['n_epochs'])
        num_units=int(best_params['num_units'])

get the performance in test set

In [ ]:
#session id
i = 0

model_lstm=LSTMDecoder(units=210,dropout=0.37,num_epochs=23)

#Fit model
model_lstm.fit(X_train_list[i],y_train_list[i])

#Get predictions
y_test_predicted_lstm=model_lstm.predict(X_test_list[i])

#Get metric of fit
R2s_lstm=get_R2(y_test_list[i],y_test_predicted_lstm)
print('session', i+1,'R2s:', R2s_lstm)